## Jupyter Notebooks for HTEM Deposition/Characterization Analysis
### Notebook 1: Basic Analysis and Plotting Tools

##### This Jupyter notebook is meant to give a demonstration of the data acquisition and basic analysis that can be done on the data residing within the HTEM API. While a number of features are available within the main HTEM website (add link here!), additional analysis is provided within each of these five notebooks. The notebooks are structured as follows:
##### 1. Basic Querying Tools
This notebook offers an introduction to the main function responsible for querying the API and the kinds of data available for analysis. It then leads a reader towards other notebooks based on particular interests and characterization specialties.
##### 2. X-Ray Diffraction: Peak Detection and Clustering Techniques
This notebook gives a demonstration of how x-ray diffraction data may be analyzed across samples and across positions. It also offers examples of how peaks could be detected and how various samples might be clustered.
##### 3. X-Ray Fluorescence: Compositional and Thickness Gradients
This notebook gives a demonstration of how x-ray fluorescence could 
##### 4. Four-Point Mapping: Sheet Resistance and Conductivity
##### 5. Ocean Optics: Spectroscopic Clustering and Band Gap Estimation

In [1]:
#Importing the necessary libraries
import jupyter_parse_api
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
color = sns.color_palette()

%matplotlib inline

##### Notice the first file imported: jupyter_parse_api. This Python file is the source for extracting large quantities of data directly from the HTEM database API. The file is structured as follows:
This function is the main function used to collect and utilize data from the HTEM database (https://api.hpc.nrel.gov/xrd/api/).  The variables that are utilized by this function are as follows:

param_list: This is the list of parameters for which we would like to
acquire data for. This list can take one of four forms:

    PDAC list: When a new sample is made, its initial name is not the
    four-digit code that the API generates. The naming convention for
    a new sample references the PDAC machine that made the sample, 
    then the five-digit number of the sample (ascending by the order
    the sample was made). There are five deposition chambers for 
    which data is readily available. The final form will be fed in as
    a list of strings that could look like this: [PDAC_COM1_00844, 
    PDAC_COM3_00132, PDAC_COM4_00588, PDAC_COM5_00021].


    Elements list: Suppose one wants to do a more general search of a
    particular material available in our database. This search tool is
    available in our database as well. One can enter a list of 
    elements in list format like so: ['Zn', 'Ni', 'O']. The elements
    need not be entered in alphabetical order. Also, this provides
    a list of samples that have ALL of the following elements present,
    not ANY of the following elements present. For example, if one 
    were to search for ['Zn','O'], then the resulting search would 
    query ZnO, SnZnO, ZnNiCoO, etc. Searching by element can be quite
    a bit more time-intensive and provide a very large quantity of 
    data from the database. One should use this setting with care.


    Sample list: When a sample is submitted to the API, it is given
    a unique 4-digit identifier we call the 'sample id'. One can 
    submit a list of samples like so: [7229,7230,7231].


    Position list: Within a single sample, there are 44 individually
    identified positions. When a gradient (compositional, temperature,
    or other) is applied across the sample, the 44 unique positions 
    allows for one to see trends across the substrate. While normally
    an entire sample would be selected for analysis, the option to 
    search based on a specific position number is also available.


Note that a list must be provided that has a continuity throughout of one of the four available types. One may not mix the available types. One should also be wary of queries that are expected to yield a large quantity of output as this may take quite a long time to load. All outputs are given as a pandas DataFrame that contains all of the data queried from the out_types command.

out_types: There are many different fields available to search 
through in this large database. In order to more efficiently process
a user's queries, a user is expected to specify which variables they 
would like to see returned to them. To have the function return each
of these fields, a list of requested fields must be supplied. The 
list of available options is quite extensive, but is enumerated here:


'xrd_all': Includes X-Ray Diffraction spectra in three parts, specifically the XRD wavelength, XRD intensity, and XRD background. Note that we commonly subtract the background from the intensity before doing any post-processing analysis. This field is specific to the positions. Below we show how this can be used to query the XRD spectra for a single position on a sample.

In [2]:
jupyter_parse_api.htem_data_collect([213809],['xrd_all'])

,213809
xrd_angle,"[52, 51.95, 51.9, 51.85, 51.8, 51.75, 51.7, 51..."
xrd_background,"[69324.921875, 69360.552083, 69644.6115449, 70..."
xrd_intensity,"[69324.921875, 67991.226562, 70765.507812, 701..."


'xrd_peak_count': A built-in algorithm does a rudimentary analysis of the peaks located within X-Ray Diffraction spectra. This field should be taken with a grain of salt since a generalized peak  fitting algorithm that accurately locates peaks in both amorphous and crystalline materials is not an easy task. The Jupyter Notebook on XRD spectra discusses this more fully. This field is specific to the positions. For example, we can query the number of peaks like so:


In [21]:
jupyter_parse_api.htem_data_collect(['Fe','Co','N'],['xrd_peak_count'])

,323357,323358,323359,323360,323361,323362,323363,323364,323365,323366,...,350692,350693,350694,350695,350696,350697,350698,350699,350700,350701
elements,"[Fe, Co, N]","[Fe, Co, N]","[Fe, Co, N]","[Fe, Co, N]","[Fe, Co, N]","[Fe, Co, N]","[Fe, Co, N]","[Fe, Co, N]","[Fe, Co, N]","[Fe, Co, N]",...,"[Fe, Co, N]","[Fe, Co, N]","[Fe, Co, N]","[Fe, Co, N]","[Fe, Co, N]","[Fe, Co, N]","[Fe, Co, N]","[Fe, Co, N]","[Fe, Co, N]","[Fe, Co, N]"
id,10118,10118,10118,10118,10118,10118,10118,10118,10118,10118,...,10461,10461,10461,10461,10461,10461,10461,10461,10461,10461
peak_count,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,13,11,1,1


'elements_req': This returns all of the elements contained in a sample. The field is specific to the X-Ray Fluorescence data, however in the absence of this the researcher's recordings at the time of deposition are set. This is specific to the sample level.

The example below shows how a sample may be queried to find out which elements are included in the sample.

In [5]:
jupyter_parse_api.htem_data_collect([6880],['elements_req'])

,6880
elements,"[Cr, O, Mg, N]"


'thickness': Another data field gathered from the X-Ray Fluorescence is the thickness of the sample at a given position. This is usually given in units of nanometers. This field is specific to the positions.

'composition': A data field gathered from the X-Ray Fluorescence that gives the composition of the sample based on the materials used to make up the sample. It comes in two parts: 'xrf_elements' lists the components in order, 'xrf_composition' lists the fractional composition (out of 1).


In [2]:
jupyter_parse_api.htem_data_collect([6880],['thickness','composition'])

,213769,213770,213771,213772,213773,213774,213775,213776,213777,213778,...,213803,213804,213805,213806,213807,213808,213809,213810,213811,213812
id,6880,6880,6880,6880,6880,6880,6880,6880,6880,6880,...,6880,6880,6880,6880,6880,6880,6880,6880,6880,6880
thickness,0.06122205,0.05765637,0.1000603,0.09353764,0.0900894,0.07305085,0.07295249,0.05700536,0.0541584,0.09602363,...,0.05304467,0.0637413,0.05435736,0.06409974,0.09040873,0.08845242,0.0878303,0.106414,0.05638395,0.0590963
xrf_compounds,"[MnO, CrO]","[MnO, CrO]","[MnO, CrO]","[MnO, CrO]","[MnO, CrO]","[MnO, CrO]","[MnO, CrO]","[MnO, CrO]","[MnO, CrO]","[MnO, CrO]",...,"[MnO, CrO]","[MnO, CrO]","[MnO, CrO]","[MnO, CrO]","[MnO, CrO]","[MnO, CrO]","[MnO, CrO]","[MnO, CrO]","[MnO, CrO]","[MnO, CrO]"
xrf_concentration,"[0, 100]","[0, 100]","[0, 100]","[0, 100]","[0, 100]","[0, 100]","[0, 100]","[0, 100]","[8.204097, 91.79591]","[0, 100]",...,"[3.782197, 96.21781]","[0, 100]","[4.255644, 95.74435]","[0, 100]","[0, 100]","[0, 100]","[0, 100]","[0, 100]","[9.655293, 90.3447]","[0, 100]"


'4pp_all': In this case, the '4pp' stands for 4-Point Probe, which  is used to take measures on conductivity, sheet resistance, etc. The 'all' in the command means to take all metrics gathered from the 4-Point analysis. This field is specific to the positions.


In [14]:
#jupyter_parse_api.htem_data_collect(['PDAC_COM4_01463'],['4pp_all'])
jupyter_parse_api.htem_data_collect([7882],['4pp_all'])

,248951,248952,248953,248954,248955,248956,248957,248958,248959,248960,...,248985,248986,248987,248988,248989,248990,248991,248992,248993,248994
fpm_conductivity,0.3634193114341162932025841986137,0.29292924124243148094851431728844,4.47274737673402570914957924280763,2.37717227753675736212458887158564,0.22047992550573278209491676832586,0.69316279194880626971525684262707,0.432195537335206983843501413360,0.33597071678179651860755589421421,0.3471396501758433411607436727469,15.51194129939478058058376303488458,...,0.62481910718763685553445355808444,0.8510825217555671468195843469584,0.45130678835545846820396208774473,0.498135420354004285548890538721,0.805476510969835712857586359882,2.65145888752424322631432481125462,2.09316557578604402891645626380324,6.251683127127825612281242713610155,0.39213539679607926878836105590828,0.403989493712580693912668007965
fpm_current_amps,"[-1e-06, -5e-07, 0, 5e-07, 1e-06]","[-1e-06, -5e-07, 0, 5e-07, 1e-06]","[-1e-06, -5e-07, 0, 5e-07, 1e-06]","[-1e-06, -5e-07, 0, 5e-07, 1e-06]","[-1e-06, -5e-07, 0, 5e-07, 1e-06]","[-1e-06, -5e-07, 0, 5e-07, 1e-06]","[-1e-06, -5e-07, 0, 5e-07, 1e-06]","[-1e-06, -5e-07, 0, 5e-07, 1e-06]","[-1e-06, -5e-07, 0, 5e-07, 1e-06]","[-1e-06, -5e-07, 0, 5e-07, 1e-06]",...,"[-1e-06, -5e-07, 0, 5e-07, 1e-06]","[-1e-06, -5e-07, 0, 5e-07, 1e-06]","[-1e-06, -5e-07, 0, 5e-07, 1e-06]","[-1e-06, -5e-07, 0, 5e-07, 1e-06]","[-1e-06, -5e-07, 0, 5e-07, 1e-06]","[-1e-06, -5e-07, 0, 5e-07, 1e-06]","[-1e-06, -5e-07, 0, 5e-07, 1e-06]","[-1e-06, -5e-07, 0, 5e-07, 1e-06]","[-1e-06, -5e-07, 0, 5e-07, 1e-06]","[-1e-06, -5e-07, 0, 5e-07, 1e-06]"
fpm_resistivity,2.7516424376399392401930634616208,3.41379370580620504270068726968865,0.22357623084230485245428874634567,0.42066787058286201924075990931392,4.53556031328574909667039219512980,1.44266254856024539384522154288827,2.313767527924308403141743103728,2.97644988104564995833933632019580,2.8806850484911496460906074618064,0.06446646365526256573090925450496,...,1.60046321966862340857132606717984,1.1749741939679996091436093099296,2.21578763227549664590036686270994,2.007486235950339105163749865536,1.241501131790854912499082008544,0.37715086011902443512561996336000,0.47774529237825400677303172980304,0.159956923546031383543138527498912,2.55013958997439430309661029575042,2.475311896876834197450830667816
fpm_sheet_resistance,23819.49797174712,29913.044951773445,4196.401291954757,6894.387241948224,39847.557861907284,12516.136527004559,23150.73122833104,29630.246621562004,29633.37397665552,1582.097736691008,...,15436.390121022241,11334.22930599264,20200.693313988722,17813.22507134784,10606.5504708876,10466.32959879552,8387.657731194553,4365.403113842568,24061.416743484722,22967.3183364648
fpm_standard_deviation,4.164833101092999,3.814583233794512,534.0561046503636,5.317292666376661,317508.19110757037,1.9883813521530795,13.20684781959755,22.201834345628967,11.09187415778378,0.26937886733226557,...,11.438736510720117,57.62697979396264,2.3822769107366946,1.9729640114467126,1.9324527599901087,2.807332841415351,6.805671574779435,2.589257510142375,1.261740403807713,3.6970001083513964
fpm_voltage_volts,"[-0.005242184429, -0.002612404831, 1.613354626...","[-0.006594991913, -0.003291915907, 6.184216447...","[-0.000595174356, -0.00016292222904, 0.0006618...","[-0.001518646559, -0.0007613062899, -1.1051301...","[-0.47322562917, -0.720797419, -0.6311775462, ...","[-0.002762759518, -0.001382940379, -2.75861554...","[-0.005127264833, -0.002570379656, -1.20260345...","[-0.006582243843, -0.003303351822, -2.94652544...","[-0.006556087544, -0.003290744182, -2.18400741...","[-0.0003501797454, -0.0001754854039, -1.136880...",...,"[-0.003426744248, -0.001716196164, -1.38809467...","[-0.002525095665, -0.00126355053, -6.854002137...","[-0.004453632499, -0.002224460457, 2.390577722...","[-0.003928857834, -0.001964636564, 1.082992516...","[-0.002332268021, -0.001160620971, 8.740535208...","[-0.002300373535, -0.001146605611, 6.9289442e-...","[-0.001835325286, -0.0009142622987, 1.05754860..."

'4pp_cond': Returns the conductivity. This field is specifc to the positions.

'4pp_sheet_res': Returns the sheet resistance. This field is 
specific to the positions.

'4pp_VC': In order to measure the conductivity and sheet 
resistance, a linear relationship between the voltage and the 
current must be established. This voltage/current pair is what
is returned. This field is specific to the positions.



In [16]:
jupyter_parse_api.htem_data_collect([7352,8864],['4pp_cond','elements_req','4pp_sheet_res','4pp_VC'])

,225965,225966,225967,225968,225969,225970,225971,225972,225973,225974,...,271850,271851,271852,271853,271854,271855,271856,271857,271858,271859
elements,"[Cu, S, Sn]","[Cu, S, Sn]","[Cu, S, Sn]","[Cu, S, Sn]","[Cu, S, Sn]","[Cu, S, Sn]","[Cu, S, Sn]","[Cu, S, Sn]","[Cu, S, Sn]","[Cu, S, Sn]",...,"[Cu, O]","[Cu, O]","[Cu, O]","[Cu, O]","[Cu, O]","[Cu, O]","[Cu, O]","[Cu, O]","[Cu, O]","[Cu, O]"
fpm_conductivity,0.65016906402741863790160,0.5677750073922451684317,0.3316822022082669879440,0.36624003805560943138053,0.4056971081856285212497,0.46850964115154917227294121969583,0.5790848071474348242994,0.3367442274722881287590,0.1014803786166474300678,0.2627939080886453519071,...,0.00630945684037931712991849744,0.0064233716784773845747321233,0.006801724317030811999987372722,0.0071963231910183546433383675,0.007482535940182987940897895605,0.00802008831387477810738859071,0.00822163914695647124398330051,0.0084238540925146495567903187361,0.00857861741431421056582312678,0.008676794249034633627930226225
fpm_current_amps,"[-1e-06, -5e-07, 1e-08, 5.1e-07, 1.01e-06]","[-1e-06, -5e-07, 1e-08, 5.1e-07, 1.01e-06]","[-1e-06, -5e-07, 1e-08, 5.1e-07, 1.01e-06]","[-1e-06, -5e-07, 1e-08, 5.1e-07, 1.01e-06]","[-1e-06, -5e-07, 1e-08, 5.1e-07, 1.01e-06]","[-1e-06, -5e-07, 1e-08, 5.1e-07, 1.01e-06]","[-1e-06, -5e-07, 1e-08, 5.1e-07, 1.01e-06]","[-1e-06, -5e-07, 1e-08, 5.1e-07, 1.01e-06]","[-1e-06, -5e-07, 1e-08, 5.1e-07, 1.01e-06]","[-1e-06, -5e-07, 1e-08, 5.1e-07, 1.01e-06]",...,"[-1e-08, -5e-09, 1e-09, 6e-09, 1.1e-08]","[-1e-08, -5e-09, 1e-09, 6e-09, 1.1e-08]","[-1e-08, -5e-09, 1e-09, 6e-09, 1.1e-08]","[-1e-08, -5e-09, 1e-09, 6e-09, 1.1e-08]","[-1e-08, -5e-09, 1e-09, 6e-09, 1.1e-08]","[-1e-08, -5e-09, 1e-09, 6e-09, 1.1e-08]","[-1e-08, -5e-09, 1e-09, 6e-09, 1.1e-08]","[-1e-08, -5e-09, 1e-09, 6e-09, 1.1e-08]","[-1e-08, -5e-09, 1e-09, 6e-09, 1.1e-08]","[-1e-08, -5e-09, 1e-09, 6e-09, 1.1e-08]"
fpm_sheet_resistance,24648.531932019992,29852.084420741965,37166.51758482369,33296.798588134516,30518.11726190095,27450.231099283916,21172.47683136402,39164.76766661611,142174.21261791425,46530.72052643268,...,2857341.091186255,2749644.877793074,2679298.9610374398,2569889.4673243873,2501968.7662616223,2433854.523112571,2406296.490823873,2386336.0724925906,2389447.8949147584,2416151.183947801
fpm_voltage_volts,"[-0.005444486609, -0.002725124987, 5.057200271...","[-0.006590606624, -0.003297005372, 6.330647297...","[-0.008200817861, -0.004100631243, 8.142590071...","[-0.007350123797, -0.003674404967, 7.623192964...","[-0.006739254575, -0.003366813993, 6.899001797...","[-0.006057734538, -0.003027748948, 6.053999055...","[-0.004674390975, -0.002337501189, 4.466776297...","[-0.008646927325, -0.00432274497, 8.274147801e...","[-0.03143656031, -0.01571447333, 0.00029308437...","[-0.01027536674, -0.005131964732, 0.0001042319...",...,"[-0.006949122499, -0.003590481553, 0.000109136...","[-0.00660204442, -0.003613772932, -8.144184887...","[-0.006414389381, -0.003474383857, 8.523796235...","[-0.006121013665, -0.003576013662, 5.005470199...","[-0.005995094815, -0.003242243879, 1.386272687...","[-0.00585608434, -0.003197994376, 7.242030338e...","[-0.005836418469, -0.003139402927, -5.15492425...","[-0.005712309155, -0.003201647109, -5.72275228...","[-0.005776038759, -0.003092003179, 1.71236189e...","[-0.005813148912, -0.003128338744, -2.73010359..."
id,7352,7352,7352,7352,7352,7352,7352,7352,7352,7352,...,8864,8864,8864,8864,8864,8864,8864,8864,8864,8864


'oo_all': Returns all data taken from Ocean Optics measurements, including UVIR, UVIT, NIRR, NIRT (described below), and the tabulated direct and indirect band gaps (calculated using MARS).


In [17]:
jupyter_parse_api.htem_data_collect([6950],['oo_all'])
#Could also be written as:
#jupyter_parse_api.htem_data_collect(['PDAC_COM4_00433'],['oo_all'])

,215700,215701,215702,215703,215704,215705,215706,215707,215708,215709,...,215734,215735,215736,215737,215738,215739,215740,215741,215742,215743
id,6950,6950,6950,6950,6950,6950,6950,6950,6950,6950,...,6950,6950,6950,6950,6950,6950,6950,6950,6950,6950
nirr_response,"[-0.000729204970412, 0.00103006651625, -0.0004...","[0.00197857385501, -0.00104265857954, 0.000518...","[-0.0022394284606, -0.00159225880634, -0.00307...","[0.000468547543278, -0.00126443244517, 0.00151...","[0.00187482591718, 0.0010769397486, 0.00169614...","[0.0006249420112, 0.000374413764803, 0.0007589...","[-0.00255189952441, -0.000597107980866, 0.0002...","[0.000418417039327, 0.00119427032769, -0.00064...","[-0.000671149464324, 0.000777009758167, 0.0007...","[-0.00213548331521, 0.00276265922002, -0.00058...",...,"[0.0022394284606, 0.000842859677505, -0.001026...","[-0.0006249420112, 0.000983193400316, -0.00071...","[0.000431035499787, 0.000404204620281, -0.0003...","[0.000430865067756, 0.00143399974331, -0.00156...","[-0.000989544438198, 0.000140333708259, 0.0004...","[-0.00187482591718, 0.00182633893564, -0.00178...","[-0.000234708204516, 0.00161390774883, -0.0004...","[-0.000156394438818, 0.00266919867136, -0.0009...","[-0.00453098816797, 0.00510403141379, 0.001339...","[-5.2131479606e-05, -0.000702525954694, -0.001..."
nirr_wavelength,"[1647.73144531, 1646.02185059, 1644.31213379, ...","[1627.20605469, 1625.49450684, 1623.78271484, ...","[1647.73144531, 1646.02185059, 1644.31213379, ...","[1647.73144531, 1646.02185059, 1644.31213379, ...","[1647.73144531, 1646.02185059, 1644.31213379, ...","[1647.73144531, 1646.02185059, 1644.31213379, ...","[1647.73144531, 1400.03735352, 1398.30822754, ...","[1508.74719238, 1507.02539062, 1505.3034668, 1...","[1639.18212891, 1637.47180176, 1635.76123047, ...","[1647.73144531, 1646.02185059, 1644.31213379, ...",...,"[1647.73144531, 1646.02185059, 1644.31213379, ...","[1647.73144531, 1646.02185059, 1644.31213379, ...","[1221.50549316, 1219.76879883, 1218.03198242, ...","[1520.79602051, 1519.07519531, 1517.35424805, ...","[1647.73144531, 1646.02185059, 1644.31213379, ...","[1647.73144531, 1646.02185059, 1644.31213379, ...","[1575.78967285, 1574.07348633, 1572.35693359, ...","[1647.73144531, 1646.02185059, 1644.31213379, ...","[1647.73144531, 1646.02185059, 1644.31213379, ...","[1647.73144531, 1646.02185059, 1644.31213379, ..."
nirt_response,"[0.551031827927, 0.543176710606, 0.54052394628...","[0.540881872177, 0.533838868141, 0.53364944458...","[0.536230266094, 0.534103453159, 0.53697103261...","[0.360765457153, 0.356717318296, 0.35560882091...","[0.335184067488, 0.335158705711, 0.33445158600...","[0.455215215683, 0.449836432934, 0.44664311409...","[0.595920741558, 0.59659320116, 0.603459954262...","[0.598173260689, 0.590921223164, 0.58612763881...","[0.594008564949, 0.586760401726, 0.58286601305...","[0.776016592979, 0.770888388157, 0.75994437933...",...,"[0.632478296757, 0.623089551926, 0.61890405416...","[0.676066637039, 0.663736581802, 0.65833610296...","[0.570394217968, 0.558967888355, 0.55627262592...","[0.548838317394, 0.541340470314, 0.53784453868...","[0.554095506668, 0.553540945053, 0.55407279729...","[0.429344952106, 0.435790330172, 0.44187244772...","[0.444103509188, 0.439690977335, 0.43585014343...","[0.698743402958, 0.690682351589, 0.68063086271...","[0.5480671525, 0.54156178236, 0.538875997066, ...","[0.547105431557, 0.538475036621, 0.53517645597..."
nirt_wavelength,"[2091.31665039, 2083.50976562, 2075.71850586, ...","[2091.31665039, 2083.50976562, 2075.71850586, ...","[1775.67456055, 1768.43054199, 1761.19763184, ...","[2091.31665039, 2083.50976562, 2075.71850586, ...","[1841.38110352, 1834.03356934, 1826.69799805, ...","[2091.31665039, 2083.50976562, 2075.71850586, ...","[1653.93676758, 1646.86315918, 1639.79846191, ...","[2091.31665039, 2083.50976562, 2075.71850586, ...","[2091.31665039, 2083.50976562, 2075.71850586, ...","[2091.31665039, 2083.50976562, 2075.71850586, ...",...,"[2091.31665039, 2083.50976562, 

'oo_uvit': Returns just the UVIT (Ultraviolet Intensity of 
Transmittance), a measurement of the Ocean Optics equipment.


'oo_uvir': Returns just the UVIR (Ultraviolet Intensity of 
Reflectance), a measurement of the Ocean Optics Equipment.


'oo_nirt': Returns just the NIRT (Near-Infrared Transmittance),
a measurement of the Ocean Optics Equipment.


'oo_nirr': Returns just the NIRR (Near-Infrared Reflectance),
a measurement of hte Ocean Optics Equipment.


'bg_all': Returns the direct and indirect band gaps from the 
materials; these are identified using a MARS technique described
in Schwarting et. al., paper to appear.


'bg_direct': Returns just the direct band gap (MARS Method)


'bg_indirect': Returns just the indirect band gap (MARS Method)


In [20]:
jupyter_parse_api.htem_data_collect([6990],['bg_direct'])
jupyter_parse_api.htem_data_collect([6990],['oo_uvit','oo_uvir'])

,216900,216901,216902,216903,216904,216905,216906,216907,216908,216909,...,216934,216935,216936,216937,216938,216939,216940,216941,216942,216943
id,6990,6990,6990,6990,6990,6990,6990,6990,6990,6990,...,6990,6990,6990,6990,6990,6990,6990,6990,6990,6990
uvir_response,"[0.283424317837, 0.335501879454, 0.36923936009...","[0.281327962875, 0.285312086344, 0.29577565193...","[0.00793825928122, 0.0168408323079, 0.02966723...","[-0.0111136501655, 0.0161958970129, 0.04615145...","[0.137417718768, 0.14887689054, 0.157924726605...","[0.236703306437, 0.243645578623, 0.26880291104...","[0.348038136959, 0.348218411207, 0.35031625628...","[0.159239292145, 0.170253649354, 0.14836846292...","[0.332104802132, 0.356785655022, 0.35301300883...","[0.00490468135104, 0.012614983134, 0.033261399...",...,"[0.333380490541, 0.354436933994, 0.35750815272...","[0.287281602621, 0.300978451967, 0.32454195618...","[0.279343336821, 0.310075163841, 0.33443424105...","[0.308601677418, 0.312890708447, 0.32484307885...","[0.113799706101, 0.120469026268, 0.11064925044...","[0.0426967032254, 0.0374983325601, 0.080612406...","[0.00425257673487, -0.000529407348949, 0.04914...","[0.000255145452684, 0.02505665645, 0.010189768...","[0.354261755943, 0.412607967854, 0.37600946426...","[0.32725673914, 0.346105545759, 0.364700078964..."
uvir_wavelength,"[1068, 1067, 1066, 1065, 1064, 1063, 1062, 106...","[1099, 1098, 1097, 1096, 1095, 1094, 1093, 109...","[1099, 1098, 1097, 1096, 1095, 1094, 1093, 109...","[1099, 1098, 1097, 1096, 1095, 1094, 1093, 109...","[1099, 1098, 1097, 1096, 1095, 1094, 1093, 109...","[1099, 1098, 1097, 1096, 1095, 1094, 1093, 109...","[1099, 1098, 1097, 1096, 1095, 1094, 1093, 109...","[1059, 1058, 1057, 1056, 1055, 1054, 1053, 105...","[1099, 1098, 1097, 1096, 1095, 1094, 1093, 109...","[1099, 1098, 1097, 1096, 1095, 1094, 1093, 109...",...,"[1099, 1098, 1097, 1096, 1095, 1094, 1093, 109...","[1099, 1098, 1097, 1096, 1095, 1094, 1093, 109...","[1099, 1098, 1097, 1096, 1095, 1094, 1093, 109...","[1099, 1098, 1097, 1096, 1095, 1094, 1093, 109...","[1075, 1074, 1073, 1072, 1071, 1070, 1069, 106...","[1099, 1098, 1097, 1096, 1095, 1094, 1093, 109...","[1099, 1098, 1097, 1096, 1095, 1094, 1093, 109...","[1099, 1098, 1097, 1096, 1095, 1094, 1093, 109...","[1083, 1082, 1081, 1080, 1079, 1078, 1077, 107...","[1099, 1098, 1097, 1096, 1095, 1094, 1093, 109..."
uvit_response,"[0.451479077339, 0.448510974646, 0.45213493704...","[0.740119814873, 0.753564178944, 0.73228383064...","[1.12079596519, 1.12585747242, 1.10667228699, ...","[0.00733855972067, 0.0187411904335, 0.01128601...","[0.171020805836, 0.156524002552, 0.15039335191...","[0.278217405081, 0.277187019587, 0.27662548422...","[0.903898000717, 0.901241719723, 0.91088765859...","[0.833735466003, 0.814296662807, 0.82230997085...","[0.723590433598, 0.723036170006, 0.71562182903...","[1.15651214123, 1.11506056786, 1.09068357944, ...",...,"[0.802879989147, 0.829966187477, 0.82677179575...","[0.794087767601, 0.804507017136, 0.77217853069...","[0.517171561718, 0.529039978981, 0.53832072019...","[0.592654407024, 0.58491820097, 0.583727180958...","[0.173127993941, 0.168966412544, 0.20498700439...","[0.00972312130034, 0.0411284826696, 0.02887126...","[0.0488353483379, 0.0603627450764, 0.026508739...","[1.10111987591, 1.1112562418, 1.10236263275, 1...","[0.593843519688, 0.618584573269, 0.61523598432...","[0.610978484154, 0.581074893475, 0.59396308660..."
uvit_wavelength,"[985, 984, 983, 982, 981, 980, 979, 978, 977, ...","[1099, 1098, 1097, 1096, 1095, 1094, 1093, 109...","[1042, 1041, 1040, 1039, 1038, 1037, 1036, 103...","[1099, 1098, 1097, 1096, 1095, 1094, 1093, 109...","[1099, 1098, 1097, 1096, 1095, 1094, 1093, 109...","[815, 814, 813, 812, 811, 810, 809, 808, 807, ...","[1039, 1038, 1037, 1036, 1035, 1034, 1033, 103...","[1099, 1098, 1097, 1096, 1095, 1094, 1093, 109...","[953, 952, 951, 950, 949, 948, 947, 946, 945, ...","[1083, 1082, 1081, 1080, 1079, 1078, 1077, 107...",...,"[1099, 1098, 1097, 1096, 1095, 

'prep_temp': Returns the temperature at which the sample was prepared in the combinatorial deposition chamber.


'prep_pressure': Returns the pressure at which the sample was prepared in the combinatorial deposition chamber.


'pdac_eval': A simple argument fed to the system that returns the PDAC number (denoting the combinatorial chamber that made the  sample) and the sample number.


'positions_list': Returns a list of all positions available for a sample.

'positions_locations': The ordering of the position ID on the sample may not match the ordering on the physical substrate in all cases. The standard for samples is that fourty-four points will be laid out in a 4x11 grid as shown below:

| 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 |
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
| 12 | 13 | 14 | 15 | 16 | 17 | 18 | 19 | 20 | 21 | 22 |
| 23 | 24 | 25 | 26 | 27 | 28 | 29 | 30 | 31 | 32 | 33 |
| 34 | 35 | 36 | 37 | 38 | 39 | 40 | 41 | 42 | 43 | 44 |

When doing examples of how a sample is physically laid out based on position on the substrate, it becomes necessary to use the 'positions_locations' field.



In [2]:
jupyter_parse_api.htem_data_collect(['Fe','Co','N'],['prep_temp','prep_pressure','pdac_eval','positions_list',\
                                                     'positions_locations'])

,323357,323358,323359,323360,323361,323362,323363,323364,323365,323366,...,350692,350693,350694,350695,350696,350697,350698,350699,350700,350701
deposition_growth_pressure_mtorr,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
deposition_initial_temp_c,300,300,300,300,300,300,300,300,300,300,...,25,25,25,25,25,25,25,25,25,25
elements,"[Fe, Co, N]","[Fe, Co, N]","[Fe, Co, N]","[Fe, Co, N]","[Fe, Co, N]","[Fe, Co, N]","[Fe, Co, N]","[Fe, Co, N]","[Fe, Co, N]","[Fe, Co, N]",...,"[Fe, Co, N]","[Fe, Co, N]","[Fe, Co, N]","[Fe, Co, N]","[Fe, Co, N]","[Fe, Co, N]","[Fe, Co, N]","[Fe, Co, N]","[Fe, Co, N]","[Fe, Co, N]"
id,10118,10118,10118,10118,10118,10118,10118,10118,10118,10118,...,10461,10461,10461,10461,10461,10461,10461,10461,10461,10461
num,2039,2039,2039,2039,2039,2039,2039,2039,2039,2039,...,2045,2045,2045,2045,2045,2045,2045,2045,2045,2045
pdac,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
position,42,43,24,25,26,27,20,21,22,23,...,19,18,31,30,37,36,35,34,33,32
position_ids,"[323372, 323389, 323376, 323400, 323373, 32336...","[323372, 323389, 323376, 323400, 323373, 32336...","[323372, 323389, 323376, 323400, 323373, 32336...","[323372, 323389, 323376, 323400, 323373, 32336...","[323372, 323389, 323376, 323400, 323373, 32336...","[323372, 323389, 323376, 323400, 323373, 32336...","[323372, 323389, 323376, 323400, 323373, 32336...","[323372, 323389, 323376, 323400, 323373, 32336...","[323372, 323389, 323376, 323400, 323373, 32336...","[323372, 323389, 323376, 323400, 323373, 32336...",...,"[350683, 350684, 350661, 350662, 350663, 35068...","[350683, 350684, 350661, 350662, 350663, 35068...","[350683, 350684, 350661, 350662, 350663, 35068...","[350683, 350684, 350661, 350662, 350663, 35068...","[350683, 350684, 350661, 350662, 350663, 35068...","[350683, 350684, 350661, 350662, 350663, 35068...","[350683, 350684, 350661, 350662, 350663, 35068...","[350683, 350684, 350661, 350662, 350663, 35068...","[350683, 350684, 350661, 350662, 350663, 35068...","[350683, 350684, 350661, 350662, 350663, 35068..."


This completes our first tutorial of the NREL HTEM materials database API. Later, we will explore tools in Jupyter Notebooks that facilitate interesting visualizations of the data.